In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("../Data/clean_data.csv")
df

,No,Price,Area,Bedrooms,WCs,District,Postdate,Description,Extracted_Price,Extracted_Area,Extracted_Bedrooms,Extracted_WCs,Price_per_sqm
0,1,13.5,44.0,NaN,NaN,quan-1,2023-11-24,"Chủ kẹt cần bán gấp nhà 2 MT Hoàng Sa, Phường ...",1.350000e+10,44.0,NaN,NaN,0.31
1,2,8.7,45.0,3.0,3.0,quan-1,2023-11-27,"Vị trí khu vực ngay đường Trần Hưng Đạo, đối ...",NaN,NaN,NaN,NaN,0.19
2,3,21.5,64.0,NaN,NaN,quan-1,2023-11-27,"Vị trí trung tâm Quận 1, khách thượng lưu và ...",NaN,NaN,NaN,NaN,0.34
3,4,9.3,40.0,4.0,4.0,quan-1,2023-11-29,"Kết cấu 1 trệt, 2 lầu, sân thượng, 3 PN, 4WC....",NaN,NaN,3.0,4.0,0.23
4,5,130.0,455.0,NaN,NaN,quan-1,2023-10-25,Diện tích 13 x 35m. | Hiện trạng Nhà cấp 4. |...,1.300000e+11,455.0,NaN,NaN,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6355,6356,26.8,200.0,5.0,5.0,thu-duc,2023-01-12,Chủ ngộp cần tiền bán gấp căn nhà Mặt Tiền ki...,2.680000e+10,200.0,5.0,5.0,0.13
6356,6357,13.5,104.7,NaN,NaN,thu-duc,2023-01-06,"Vị trí Hẻm số 652 Ouốc Lộ 13, Hiệp Bình Phước ...",1.350000e+10,NaN,NaN,NaN,0.13
6357,6358,6.8,68.0,NaN,NaN,thu-duc,2023-01-07,"Nhà đẹp 3 tấm , hẻm xe hơi 5m , P. Trường Thọ...",6.800000e+09,NaN,NaN,NaN,0.10
6358,6359,6.5,62.5,NaN,NaN,thu-duc,2023-01-05,"Cần bán nhà đường số 2, Trường Thọ, Thủ Đức. ...",6.500000e+09,NaN,NaN,NaN,0.10


In [3]:
def extract_property_info(text):
    # Biến chứa các thông tin cần trích xuất
    info = {
        'price': None,
        'address': None,
        'area': None,
        'bedrooms': None,
        'toilets': None,
        'floors': None,
        'other_features': None
    }
    
    if isinstance(text, str):
        # Trích xuất giá
        price_match = re.search(r'Giá ([\d\.]+) tỷ', text)
        if price_match:
            info['price'] = float(price_match.group(1))
        
        # Trích xuất địa chỉ
        address_match = re.search(r'Vị trí (.+?) \|', text)
        if address_match:
            info['address'] = address_match.group(1).strip()
        
        # Trích xuất diện tích
        area_match = re.search(r'Diện tích (\d+\.?\d*)m2', text)
        if area_match:
            info['area'] = float(area_match.group(1))
        
        # Trích xuất số phòng ngủ
        bedrooms_match = re.search(r'(\d+) phòng ngủ', text)
        if bedrooms_match:
            info['bedrooms'] = int(bedrooms_match.group(1))
        
        # Trích xuất số toilet
        toilets_match = re.search(r'(\d+) toilet', text)
        if toilets_match:
            info['toilets'] = int(toilets_match.group(1))
        
        # Trích xuất số tầng
        floors_match = re.search(r'(\d+) tầng', text)
        if floors_match:
            info['floors'] = int(floors_match.group(1))
        
        # Trích xuất các đặc điểm khác
        other_features_match = re.search(r'(\d+ tầng, .+?) \|', text)
        if other_features_match:
            info['other_features'] = other_features_match.group(1).strip()
    
    return info

# Đọc dữ liệu từ file CSV
df = pd.read_csv('../Data/clean_data.csv')

# Tạo các cột mới từ thông tin trích xuất
df['price'] = df['Description'].apply(lambda x: extract_property_info(x)['price'] if pd.notnull(x) else None)
df['address'] = df['Description'].apply(lambda x: extract_property_info(x)['address'] if pd.notnull(x) else None)
df['area'] = df['Description'].apply(lambda x: extract_property_info(x)['area'] if pd.notnull(x) else None)
df['bedrooms'] = df['Description'].apply(lambda x: extract_property_info(x)['bedrooms'] if pd.notnull(x) else None)
df['toilets'] = df['Description'].apply(lambda x: extract_property_info(x)['toilets'] if pd.notnull(x) else None)
df['floors'] = df['Description'].apply(lambda x: extract_property_info(x)['floors'] if pd.notnull(x) else None)
df['other_features'] = df['Description'].apply(lambda x: extract_property_info(x)['other_features'] if pd.notnull(x) else None)

# Chỉ lưu các cột đã trích xuất vào file CSV mới
extracted_columns = df[['price', 'address', 'area', 'bedrooms', 'toilets', 'floors', 'other_features']]
extracted_columns.to_csv('../Data/extracted_data_description.csv', index=False)

print("Extraction complete. Results saved to '../Data/extracted_data_description.csv'.")


Extraction complete. Results saved to '../Data/extracted_data_description.csv'.
